## Project 2
### Census Data Enrichment File
###### As at August 8, 2019

Upload and preparation of enrichment datasets.

### Import Standard Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlrd

### Define Import Functions

Define import functions

In [2]:
def import_csv(input_file_name):
    """
    Define function to import csv file
    """
    return pd.read_csv(input_file_name)

def import_xlsx(input_file_name, worksheet_name):
    """
    Define function to import xlsx file
    """
    return pd.read_excel(input_file_name)

def df_properties(pd_dataframe):
    """
    List basic properties of pandas dataframe
    """
    shape = pd_dataframe.shape
    head = pd_dataframe.head()
    de_types = pd_dataframe.dtypes
    # Check all variables for NaNs (missing values)
    missing_de_vals = pd_dataframe.isnull().sum()
    text_out = "\nDataframe shape:\t"+ str(shape) +\
                "\n\nDataframe head:\n" + str(head) +\
                "\n\nData element types:\n" + str(de_types) +\
                "\n\nMissing values by data element:\n" + str(missing_de_vals)
    return text_out


### Define Utility Functions


In [3]:
def rem_dups_in_list(list_in):
    """
    Remove any duplicate values in a listb
    """
    return list(dict.fromkeys(list_in))


### Define Error Check Functions

In [4]:
def check_nan_cols(dataframe):
    """
    Identify columns containing NaNs
    """
    cols_w_nan = dataframe.isna().any()
    return cols_w_nan
    
def cols_nan(dataframe):
    """
    List of cols with NaNs
    """
    col_names = dataframe.columns[dataframe.isna().any()].tolist()
    return col_names

def cols_nan_sub(dataframe):
    """
    Select cols with NaN
    """
    nan_sub = dataframe.loc[:, dataframe.isna().any()]
    return nan_sub

def rows_nan_sub(dataframe, columns):
    """
    Select only rows where column is NaN
    """
    nan_sub = dataframe[dataframe[columns].isnull()]
    return nan_sub

### Define Chart Functions


In [5]:
def plot_chart(title, x_label, y_label):
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()
    return

def line_plot(x_val_list, y_val_list, title, x_label, y_label):
    plt.plot(x_val_list, y_val_list)
    plot_chart(title, x_label, y_label)
    return

def scatter_plot(x_val_list, y_val_list, title, x_label, y_label):
    plt.scatter(x_val_list, y_val_list)
    plot_chart(title, x_label, y_label)
    return

def hist_plot(data_list, bin_count, title, x_label, y_label):
    plt.hist(data_list, bins = bin_count)
    plot_chart(title, x_label, y_label)
    return

def box_plot(data_list, title, x_label):
    plt.boxplot(data_list)
    y_label = "Value"
    plt.yticks(range(min(data_list), max(data_list)))
    plot_chart(title, x_label, y_label)
    return

def bar_chart(cat_names_list, cat_values_list, title, x_label, y_label):
    y_positions = range(len(cat_names_list))
    plt.bar(y_positions, cat_values_list)
    plt.xticks(y_positions, cat_names_list)
    plot_chart(title, x_label, y_label)
    return


### Import Datasets

#### Import income and population data set

In [6]:
# Import income and population dataset
dat_IncByZIP = pd.read_excel("MedianZIP-3.xlsx", "nation")
#dat_IncByZIP = import_xlsx("MedianZIP-3.xlsx", "nation")

# Rename data elements
dat_IncByZIP = dat_IncByZIP.rename(columns = {"Zip":"ZIP",
                                              "Median":"Income_Median",
                                              "Mean":"Income_Mean",
                                              "Pop":"Population"})

# View initial dataframe properties
print(df_properties(dat_IncByZIP))

# Identify and remove rows with NaNs
dat_IncByZIP = dat_IncByZIP.dropna() # drops NaN

# Identify index of all rows containing cells with only a "."
errors = dat_IncByZIP[dat_IncByZIP == "."].dropna(how="all")
errors_idx_list = errors.index.tolist() 
dat_IncByZIP.drop(errors_idx_list, axis="rows", inplace=True) # Drop erroneous rows

# Change Income_Mean to float
dat_IncByZIP.Income_Mean = dat_IncByZIP.Income_Mean.astype("float64")

print(df_properties(dat_IncByZIP))



Dataframe shape:	(32634, 4)

Dataframe head:
    ZIP  Income_Median Income_Mean  Population
0  1001     56662.5735     66687.8       16445
1  1002     49853.4177     75062.6       28069
2  1003     28462.0000       35121        8491
3  1005     75423.0000       82442        4798
4  1007     79076.3540       85802       12962

Data element types:
ZIP                int64
Income_Median    float64
Income_Mean       object
Population         int64
dtype: object

Missing values by data element:
ZIP              0
Income_Median    0
Income_Mean      0
Population       0
dtype: int64

Dataframe shape:	(32627, 4)

Dataframe head:
    ZIP  Income_Median  Income_Mean  Population
0  1001     56662.5735   66687.7509       16445
1  1002     49853.4177   75062.6343       28069
2  1003     28462.0000   35121.0000        8491
3  1005     75423.0000   82442.0000        4798
4  1007     79076.3540   85801.9750       12962

Data element types:
ZIP                int64
Income_Median    float64
Income_Mea

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


#### Import ZIP to FIPS State/County mapping

In [7]:
# Import ZIP to FIPS State/County mapping
dat_FIPStoZIP = import_csv("ZIP-COUNTY-FIPS_2017-06.csv")
# Sort dataframe by ZIP code
dat_FIPStoZIP.sort_values(by=["ZIP"], inplace=True)
# Rename data elements
dat_FIPStoZIP = dat_FIPStoZIP.rename(columns = {"COUNTYNAME":"County_Name",
                                                "STCOUNTYFP":"FIPS_County",
                                                "CLASSFP":"FIPS_Class"})

print(df_properties(dat_FIPStoZIP))

# Check unique FIPS state/county pairs
unique_fips_state_county = dat_FIPStoZIP.groupby(["FIPS_County", "FIPS_Class"]).ngroups
print("\nUnique FIPS state/county pairs:", unique_fips_state_county)

# Check unique ZIPs
unique_zip = dat_FIPStoZIP.groupby(["ZIP"]).ngroups
print("\nUnique ZIP codes in FIPS file:", unique_zip)

# Check unique FIPS state, FIPS county and ZIP triplets
unique_fips_zip = dat_FIPStoZIP.groupby(["ZIP", "FIPS_County", "FIPS_Class"]).ngroups
print("\nUnique FIPS state, FIPS county and ZIP triplets:", unique_fips_zip)

# Retain only unique FIPS state, FIPS county and ZIP triplets
## Map ZIP code to only one FIPS state/county pair
dat_ZIPtoFIPS = dat_FIPStoZIP
dat_ZIPtoFIPS = dat_ZIPtoFIPS.drop_duplicates(subset=["ZIP"],keep="first")
print(df_properties(dat_ZIPtoFIPS))



Dataframe shape:	(52889, 5)

Dataframe head:
       ZIP          County_Name STATE  FIPS_County FIPS_Class
32368  501       Suffolk County    NY        36103         H1
52575  601   Adjuntas Municipio    PR        72001         H1
52771  601      Ponce Municipio    PR        72113         H1
52577  602  Aguadilla Municipio    PR        72005         H1
52576  602     Aguada Municipio    PR        72003         H1

Data element types:
ZIP             int64
County_Name    object
STATE          object
FIPS_County     int64
FIPS_Class     object
dtype: object

Missing values by data element:
ZIP            0
County_Name    0
STATE          0
FIPS_County    0
FIPS_Class     0
dtype: int64

Unique FIPS state/county pairs: 3223

Unique ZIP codes in FIPS file: 39456

Unique FIPS state, FIPS county and ZIP triplets: 52889

Dataframe shape:	(39456, 5)

Dataframe head:
       ZIP          County_Name STATE  FIPS_County FIPS_Class
32368  501       Suffolk County    NY        36103         H1
5257

#### Import beneficiary summary file

Import beneficiary summary file to understand how location is codified. Specifically, review data elements SP_STATE_CODE and BENE_COUNTY_CD.

##### Data dictionary information for SP_STATE_CODE
- Column: 7 
- DESYNPUF: State Code
- BEN-7.
- Label: DESYNPUF: State Code
- Variable Name: SP_STATE_CODE
- Type: Char
- File: 2008 Beneficiary Summary File
- Variable: STATE_CODE
- Label: The state of residence of the beneficiary and is based on the mailing address used for cash benefits or the mailing address used for other purposes (for example, premium billing). This information is maintained from change of address notices sent in by the beneficiaries, and is appended to the record at time of processing in central office. The coding system is the SSA system, not the Federal Information Processing Standard (FIPS).
- Coding scheme: Others = Puerto Rico, Virgin Islands, Africa, Asia Or California; Institutional Provider Of Services (IPS) Only, Canada & Islands, Central America And West Indies, Europe, Mexico, Oceania, Philippines, South America, U.S. Possessions, American Samoa, Guam, Saipan Or Northern Marianas, Texas; Institutional Provider Of Services (IPS) Only, Northern Marianas, Guam, Unknown.

##### Data dictionary information for BENE_COUNTY_CD
- Column: 8
- DESYNPUF: County Code
- Label: DESYNPUF: County Code
- Variable Name: BENE_COUNTY_CD
- Type: Char
- File: 2008 Beneficiary Summary File
- Variable: BENE_COUNTY_CD
- Label: This code specifies the SSA code for the county of residence of the beneficiary. Each state has a series of codes beginning with '000' for each county within that state. Certain cities within that state have their own code. County codes must be combined with state codes in order to locate the specific county. The coding system is the SSA system, not the federal information processing standard (FIPS).
- Note: This variable was imputed/suppressed/coarsened as part of disclosure treatment. Analyses using this variable should be interpreted with caution. Analytic inferences to the Medicare population should not be made when using this variable.


In [8]:
# Import beneficiary summary file
dat_ben = import_csv("DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv")
dat_ben = dat_ben[["BENE_COUNTY_CD", "SP_STATE_CODE"]]
print(df_properties(dat_ben))

# Rename data elements
dat_ben = dat_ben.rename(columns = {"SP_STATE_CODE":"SSA_State_Code",
                                    "BENE_COUNTY_CD":"SSA_County"})

# Retain only subset of unique pairs
SSA_county_state_pairs = dat_ben.drop_duplicates(subset=["SSA_State_Code","SSA_County"],keep=False)
#print(df_properties(SSA_county_state_pairs))

# Reorder columns
new_order = [1,0]
SSA_county_state_pairs = SSA_county_state_pairs[SSA_county_state_pairs.columns[new_order]]

# Sort dataframe by state, then county
SSA_county_state_pairs.sort_values(by=["SSA_State_Code", "SSA_County"], inplace=True)
print(df_properties(SSA_county_state_pairs))

# Check unique SSA county/state pairs
unique_ssa_state_county = SSA_county_state_pairs.groupby(["SSA_State_Code", "SSA_County"]).ngroups
print("\nUnique SSA state/county pairs:", unique_ssa_state_county)


Dataframe shape:	(116352, 2)

Dataframe head:
   BENE_COUNTY_CD  SP_STATE_CODE
0             950             26
1             230             39
2             280             39
3             290              6
4             590             52

Data element types:
BENE_COUNTY_CD    int64
SP_STATE_CODE     int64
dtype: object

Missing values by data element:
BENE_COUNTY_CD    0
SP_STATE_CODE     0
dtype: int64

Dataframe shape:	(113, 2)

Dataframe head:
        SSA_State_Code  SSA_County
28747                1         310
69246                2          10
42131                2          40
48363                2          80
109774               2         190

Data element types:
SSA_State_Code    int64
SSA_County        int64
dtype: object

Missing values by data element:
SSA_State_Code    0
SSA_County        0
dtype: int64

Unique SSA state/county pairs: 113


In [9]:
# Import SSA state code table per data dictionary for beneficiary summary file
dat_ssa_state = import_csv("BenStateTable.csv")
dat_ssa_state = dat_ssa_state[["Code", "Label", "State_Name"]]

# Rename data elements
dat_ssa_state = dat_ssa_state.rename(columns = {"Code":"SSA_State_Code",
                                                "Label":"State"})
print(df_properties(dat_ssa_state))



Dataframe shape:	(52, 3)

Dataframe head:
   SSA_State_Code State  State_Name
0               1    AL     Alabama
1               2    AK      Alaska
2               3    AZ     Arizona
3               4    AR    Arkansas
4               5    CA  California

Data element types:
SSA_State_Code     int64
State             object
State_Name        object
dtype: object

Missing values by data element:
SSA_State_Code    0
State             0
State_Name        0
dtype: int64


#### Import FIPS crosswalk

- Need to do a two step crosswalk in order to map to zip codes
- Import the following files:
 - BenStateTable.csv
 - ssa_fips_state_county2011.csv

Source: https://www.nber.org/data/ssa-fips-state-county-crosswalk.html (2011 County Crosswalk)

In [10]:
# Import SSA to FIPS crosswalk
dat_xwalk_SSAtoFIPS = import_csv("ssa_fips_state_county2011.csv")
dat_xwalk_SSAtoFIPS = dat_xwalk_SSAtoFIPS[["county","state","ssacounty","fipscounty","ssastate","fipsstate"]]
#print(df_properties(dat_xwalk_SSAtoFIPS))
# Remove the first row containing NaN observations
dat_xwalk_SSAtoFIPS = dat_xwalk_SSAtoFIPS.drop([0], axis=0)

# Change ssacounty, ssastate, fipscounty, and fipsstate to int
dat_xwalk_SSAtoFIPS.ssacounty = dat_xwalk_SSAtoFIPS.ssacounty.astype("int64")
dat_xwalk_SSAtoFIPS.ssastate = dat_xwalk_SSAtoFIPS.ssastate.astype("int64")
dat_xwalk_SSAtoFIPS.fipscounty = dat_xwalk_SSAtoFIPS.fipscounty.astype("int64")
dat_xwalk_SSAtoFIPS.fipsstate = dat_xwalk_SSAtoFIPS.fipsstate.astype("int64")
print(df_properties(dat_xwalk_SSAtoFIPS))



Dataframe shape:	(3273, 6)

Dataframe head:
    county state  ssacounty  fipscounty  ssastate  fipsstate
1  AUTAUGA    AL       1000        1001         1          1
2  BALDWIN    AL       1010        1003         1          1
3  BARBOUR    AL       1020        1005         1          1
4     BIBB    AL       1030        1007         1          1
5   BLOUNT    AL       1040        1009         1          1

Data element types:
county        object
state         object
ssacounty      int64
fipscounty     int64
ssastate       int64
fipsstate      int64
dtype: object

Missing values by data element:
county        0
state         0
ssacounty     0
fipscounty    0
ssastate      0
fipsstate     0
dtype: int64


In [11]:
dat_xwalk_FIPStoSSA = dat_xwalk_SSAtoFIPS[["fipsstate", "fipscounty"]]
# Check unique FIPS state/county pairs
dat_xwalk_FIPStoSSA.groupby(["fipsstate", "fipscounty"]).ngroups
#print("\nUnique FIPS state/county pairs:\n", dat_xwalk_FIPStoSSA)

3272

### Create SSA state/county, to FIPS pairs to ZIP code crosswalk

Develop crosswalk for ZIP codes to SSA using two step process
Need to do a two step crosswalk in order to map to zip codes
Create crosswalk for SSA State/County pairs to ZIP code using FIPS codes

In [12]:
## Step 1
# Use an inner join to create an enriched dataframe listing all unique ZIP codes and corresponding
# unique identifier codes (i.e., FIPS state/county) and associated data elements
df_aug_step1 = pd.merge(dat_IncByZIP, dat_ZIPtoFIPS, on="ZIP", how='inner')


In [13]:
# Reorder columns
new_order = [0,5,6,7,4,1,2,3]
df_aug_step1 = df_aug_step1[df_aug_step1.columns[new_order]]
#print(df_properties(df_aug_step1))

# Sort dataframe by FIPS_State_County
df_aug_step1.sort_values(by=["FIPS_County", "ZIP"], inplace=True)
print(df_properties(df_aug_step1))

## Step 2
# Use an inner join to create an enriched dataframe listing all unique ZIP codes and corresponding
# unique identifier codes (i.e., FIPS state/county) and associated data elements
print(df_properties(dat_xwalk_SSAtoFIPS))
# Check unique fipscounty
unique_FIPS_county = dat_xwalk_SSAtoFIPS.groupby(["fipscounty"]).ngroups
print("\nUnique fipscounty:\n", unique_FIPS_county)
# Missmatch between unique fipscounty and number of rows in dataframe
# Select all duplicate rows based on one column
## [TBU] The next command should print two rows but it is only printing 1
duplicateRowsDF = dat_xwalk_SSAtoFIPS[dat_xwalk_SSAtoFIPS.duplicated(["fipscounty"])]
print("Duplicate rows based on fipscounty column are:", duplicateRowsDF, sep='\n')
 
dups_fipscounty = dat_xwalk_SSAtoFIPS.loc[dat_xwalk_SSAtoFIPS["fipscounty"] == 6037]
# View rows with fipscounty = 6037
print("\n\nDouble Check\nDuplicate rows based on fipscounty column are:")
print(dups_fipscounty)

# Note: We need a one-to-one mapping of FIPS to SSA county codes in order to tie back to ZIP codes
# Given there is only one entry for which a single FIPS county code ties to two SSA county codes,
# adjust the SSA county codes to 1 value.
# Specifically, we need to map SSA county codes 5200 and 5210 to a single value, say 5205
# Check whether SSA county code 5205 exists
#print(dat_xwalk_SSAtoFIPS.loc[dat_xwalk_SSAtoFIPS["ssacounty"] == 5205])
print("\n\nCheck SSA county code 5205:")
check_ssacounty = (5205 in dat_xwalk_SSAtoFIPS["ssacounty"])
print(check_ssacounty)
print("\n\n")



Dataframe shape:	(32529, 8)

Dataframe head:
         ZIP STATE  FIPS_County FIPS_Class     County_Name  Income_Median  \
11256  36003    AL         1001         H1  Autauga County     40078.0000   
11258  36006    AL         1001         H1  Autauga County     49031.6186   
11290  36051    AL         1001         H1  Autauga County     47114.9461   
11296  36067    AL         1001         H1  Autauga County     52862.8013   
11308  36091    AL         1001         H1  Autauga County     35959.0813   

       Income_Mean  Population  
11256   45819.0000        1975  
11258   51471.3930        1313  
11290   54316.3780        2304  
11296   65309.9346       26954  
11308   45032.2795        3648  

Data element types:
ZIP                int64
STATE             object
FIPS_County        int64
FIPS_Class        object
County_Name       object
Income_Median    float64
Income_Mean      float64
Population         int64
dtype: object

Missing values by data element:
ZIP              0
STATE 

In [14]:
# OK to map to 5205
dat_xwalk_SSAtoFIPS.ssacounty = dat_xwalk_SSAtoFIPS.ssacounty.replace(to_replace=[5200,5210],value=5205)
print("\nMapped values")
print(dat_xwalk_SSAtoFIPS.loc[dat_xwalk_SSAtoFIPS["fipscounty"] == 6037])
print("\nCheck data frame")
print(df_properties(dat_xwalk_SSAtoFIPS))



Mapped values
          county state  ssacounty  fipscounty  ssastate  fipsstate
209  LOS ANGELES    CA       5205        6037         5          6
210  LOS ANGELES    CA       5205        6037         5          6

Check data frame

Dataframe shape:	(3273, 6)

Dataframe head:
    county state  ssacounty  fipscounty  ssastate  fipsstate
1  AUTAUGA    AL       1000        1001         1          1
2  BALDWIN    AL       1010        1003         1          1
3  BARBOUR    AL       1020        1005         1          1
4     BIBB    AL       1030        1007         1          1
5   BLOUNT    AL       1040        1009         1          1

Data element types:
county        object
state         object
ssacounty      int64
fipscounty     int64
ssastate       int64
fipsstate      int64
dtype: object

Missing values by data element:
county        0
state         0
ssacounty     0
fipscounty    0
ssastate      0
fipsstate     0
dtype: int64


In [15]:
# Remove duplicate FIPS/SSA mapping
# Retain only subset of unique pairs
dat_xwalk_SSAtoFIPS = dat_xwalk_SSAtoFIPS.drop_duplicates()
print("\nCheck that dups removed")
print(df_properties(dat_xwalk_SSAtoFIPS))



Check that dups removed

Dataframe shape:	(3272, 6)

Dataframe head:
    county state  ssacounty  fipscounty  ssastate  fipsstate
1  AUTAUGA    AL       1000        1001         1          1
2  BALDWIN    AL       1010        1003         1          1
3  BARBOUR    AL       1020        1005         1          1
4     BIBB    AL       1030        1007         1          1
5   BLOUNT    AL       1040        1009         1          1

Data element types:
county        object
state         object
ssacounty      int64
fipscounty     int64
ssastate       int64
fipsstate      int64
dtype: object

Missing values by data element:
county        0
state         0
ssacounty     0
fipscounty    0
ssastate      0
fipsstate     0
dtype: int64


In [16]:
# Remove duplicate FIPS from mapping
# Retain only subset of unique pairs
dat_xwalk_SSAtoFIPS = dat_xwalk_SSAtoFIPS.drop_duplicates(subset=["fipscounty","ssacounty"],keep=False)
print("\nCheck that dups removed")
print(df_properties(dat_xwalk_SSAtoFIPS))


dups_fipscounty = dat_xwalk_SSAtoFIPS.loc[dat_xwalk_SSAtoFIPS["fipscounty"] == 6037]
# View rows with fipscounty = 6037
print("\n\nDouble Check\nDuplicate rows based on fipscounty column are:")
print(dups_fipscounty)



Check that dups removed

Dataframe shape:	(3272, 6)

Dataframe head:
    county state  ssacounty  fipscounty  ssastate  fipsstate
1  AUTAUGA    AL       1000        1001         1          1
2  BALDWIN    AL       1010        1003         1          1
3  BARBOUR    AL       1020        1005         1          1
4     BIBB    AL       1030        1007         1          1
5   BLOUNT    AL       1040        1009         1          1

Data element types:
county        object
state         object
ssacounty      int64
fipscounty     int64
ssastate       int64
fipsstate      int64
dtype: object

Missing values by data element:
county        0
state         0
ssacounty     0
fipscounty    0
ssastate      0
fipsstate     0
dtype: int64


Double Check
Duplicate rows based on fipscounty column are:
          county state  ssacounty  fipscounty  ssastate  fipsstate
209  LOS ANGELES    CA       5205        6037         5          6


In [17]:
# Execute inner join of df_aug_step1 and dat_xwalk_SSAtoFIPS based on FIPS county code
df_aug_step2 = pd.merge(df_aug_step1, dat_xwalk_SSAtoFIPS, left_on="FIPS_County", right_on="fipscounty", how="inner")
print(df_properties(df_aug_step2))

# Revised mapping 
master_FIPS_SSA_ZIP_map = df_aug_step2

# Create new column with new variable, total_inc_zip (to be used later for aggregation by SSA/FIPS)
master_FIPS_SSA_ZIP_map["total_inc_zip"] = master_FIPS_SSA_ZIP_map.apply(lambda row: row.Income_Mean*row.Population, axis=1)
print(df_properties(master_FIPS_SSA_ZIP_map))

# Change variable type of fipscounty to int64 in order to be able to proceed with join
master_FIPS_SSA_ZIP_map.total_inc_zip = master_FIPS_SSA_ZIP_map.total_inc_zip.astype("float64")
print("\n\nPost Update")
print(df_properties(master_FIPS_SSA_ZIP_map))




Dataframe shape:	(32529, 14)

Dataframe head:
     ZIP STATE  FIPS_County FIPS_Class     County_Name  Income_Median  \
0  36003    AL         1001         H1  Autauga County     40078.0000   
1  36006    AL         1001         H1  Autauga County     49031.6186   
2  36051    AL         1001         H1  Autauga County     47114.9461   
3  36067    AL         1001         H1  Autauga County     52862.8013   
4  36091    AL         1001         H1  Autauga County     35959.0813   

   Income_Mean  Population   county state  ssacounty  fipscounty  ssastate  \
0   45819.0000        1975  AUTAUGA    AL       1000        1001         1   
1   51471.3930        1313  AUTAUGA    AL       1000        1001         1   
2   54316.3780        2304  AUTAUGA    AL       1000        1001         1   
3   65309.9346       26954  AUTAUGA    AL       1000        1001         1   
4   45032.2795        3648  AUTAUGA    AL       1000        1001         1   

   fipsstate  
0          1  
1          1  


### Aggregate income and population totals by county

Aggregagate master_FIPS_SSA_ZIP_map to summarize income and population by unique SSA and FIPS codes


In [18]:
master_census = master_FIPS_SSA_ZIP_map
master_census.columns = master_census.columns.get_level_values(0)
#print(master_census.columns)

# Retain only necessary columns
master_census = master_census[["ZIP", "STATE", "County_Name", \
                               "ssastate", "ssacounty", \
                               "fipsstate", "fipscounty", \
                               "Population", "total_inc_zip"]]

master_census = pd.merge(master_census, dat_ssa_state, 
                         left_on="STATE", right_on="State", how="inner")

# Clean up column names
master_census = master_census.rename(columns = {"STATE":"state",
                                                "State_Name":"state_name",
                                                "County_Name":"county", 
                                                "Population":"population",
                                                "total_inc_zip":"income"})

# print(df_properties(master_census))

master_census.columns = master_census.columns.get_level_values(0)
# print("master census\n", master_census.columns)

master_census_by_zip = master_census
print("master census zip\n", master_census_by_zip.columns)
master_census_by_zip.to_csv(r"C:\Users\Jelena\W200_python\!! EBP\assignments_upstream_summer19-master\project_2\master_census_zip_final.csv")

master_census_by_SSA_FIPS = master_census[["state", "state_name", "county", 
                                           "ssastate", "ssacounty", 
                                           "fipsstate", "fipscounty", 
                                           "population", "income"]]
# print("master census SSA/FIPS\n", master_census_by_SSA_FIPS.columns)

master_col_groupby = ["state", "state_name", "county", 
                      "fipscounty", "fipsstate", 
                      "ssacounty", "ssastate"]

# Retain only relevant columns for summary frame
# Create summary data frame    
master_census_by_SSA_FIPS = master_census_by_SSA_FIPS.groupby(master_col_groupby).agg({"population":"sum","income":"sum"})

master_census_by_SSA_FIPS.to_csv(r"C:\Users\Jelena\W200_python\!! EBP\assignments_upstream_summer19-master\project_2\master_census_by_SSA_FIPS.csv")

# Double check master_census dataframe
print(df_properties(master_census_by_SSA_FIPS))

master census zip
 Index(['ZIP', 'state', 'county', 'ssastate', 'ssacounty', 'fipsstate',
       'fipscounty', 'population', 'income', 'SSA_State_Code', 'State',
       'state_name'],
      dtype='object')

Dataframe shape:	(3101, 2)

Dataframe head:
                                                                                     population  \
state state_name county                     fipscounty fipsstate ssacounty ssastate               
AK    Alaska     Aleutians East Borough     2013       2         2013      2               3138   
                 Aleutians West Census Area 2016       2         2016      2               5362   
                 Anchorage Municipality     2020       2         2020      2             291901   
                 Bethel Census Area         2050       2         2050      2              17001   
                 Bristol Bay Borough        2060       2         2060      2               1057   

                                                       